# Reproduction of Experiments on Dataset D14

## Read data

In [1]:
import sys 
import os
sys.path.insert(0, os.path.join("..", "src"))
import pandas as pd
%matplotlib inline

names = ["Sex", "Length", "Diameter", "Height", "Whole weight", "Shucked weight", "Viscera weight", "Shell weight", "Rings"] 

path = 'http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data'
df = pd.read_csv(path, header=None, names=names)
df

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [2]:
# preprocess categorical column
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

df["Sex"] = label_encoder.fit_transform(df["Sex"])
df

,Sex,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Rings
0,2,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,2,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,2,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,1,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,0,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,2,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,2,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,0,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [3]:
label_column = "Rings"

minority_labels = [13]
majority_labels = df[label_column].value_counts().drop(minority_labels).index.tolist() # all except 13

df_filtered = df[df[label_column].isin(majority_labels + minority_labels)].copy()

df_filtered.loc[df_filtered[label_column].isin(majority_labels), label_column] = 1
df_filtered.loc[df_filtered[label_column].isin(minority_labels), label_column] = 0

X = df_filtered.drop(label_column, axis=1)
y = df_filtered[label_column]

In [4]:
y.value_counts()

1    3974
0     203
Name: Rings, dtype: int64

# Classification

In [5]:
from sklearn.model_selection import ParameterGrid, KFold, GridSearchCV
from sklearn.metrics import roc_auc_score
import numpy as np

> **Note:** In the following, we use a custom piece of code for the grid search and cross-validation. We do this to enforce that the augmentation/oversampling step is only applied to training data and the test data is actually kept separate. For the baseline model, this means that we could also simply make use of the functions provided by sklearn instead of our custom code. However, in order to make the results comparable, we use the same code in each case.

In [6]:
kf = KFold(n_splits=5, shuffle=True, random_state=7018321)

## Random Forest

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
rf_param_grid = {
    'n_estimators': [50, 100, 200, 400, 600],
    'max_depth': [None, 4, 6, 10, 20, 30, 50, 80, 100],
}

# compute all combinations of parameters
combination_dicts = list(ParameterGrid(rf_param_grid))

### Baseline (no augmentation)

In [ ]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    #print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        
    for param_comb in combination_dicts:
        clf = RandomForestClassifier(random_state=19231823, **param_comb)
        clf.fit(X_train, y_train)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

In [10]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'max_depth': 6, 'n_estimators': 600}
Best AUC: 0.7395650202089131


To verify that our cross-validation code works as expected, we can compare to a regular sklearn grid search here:

In [11]:
baseline_search = GridSearchCV(RandomForestClassifier(random_state=19231823), rf_param_grid, scoring="roc_auc")
baseline_search.fit(X, y)
baseline_search.best_score_

0.7195691823899371

We can see that the best model found by a sklearn grid search performs similarly well to our best model, which indicates that the code works as expected.

### With CFA

In [9]:
from cfa import Iterative_CFA

For CFA, we try two different approaches, since it is not entirely clear from the paper what the authors actually did.
1. The first approach strictly follows the pseudo-code and description given in section 3.2 of the paper. 
2. In an earlier section of the paper, the authors say that _"the class of [a] new [synthetic counterfactual] instance needs to be verified by the underlying ML model."_ This means that we use some ML model trained on the data (without CFA) to assign a class to a new synthetic counterfactual, and only keep those that were classified to be a minority instance. This seems to lead to much more reasonable synthetic counterfactuals (see visualizations of the algorithm in exp001) but, at the same time, often means that the algorithm is unable to produce a fully balanced dataset (since it may terminate early).

> **Note:** We also use a different tolerance level of 50% here, since the 10%-threshold proposed by the authors does not yield any "good" native counterfactuals, which makes the algorithm unusable.

#### Approach 1: No verification 

In [10]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    X_train_augmented, y_train_augmented = Iterative_CFA(X_train, 
                                                         y_train, 
                                                         stddev_percent=50, 
                                                         verify_with_baseline_model=False,
                                                         visualize_with_pca=False)
    
    for param_comb in combination_dicts:
        clf = RandomForestClassifier(random_state=19231823, **param_comb)
        clf.fit(X_train_augmented, y_train_augmented)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

Fold 0:
Data distribution before CFA:
	Majority (1): 3183, Minority (label 0): 158
Number of 'good' native counterfactuals in data: 1378
Data distribution after iteration 0:
	Majority (1): 3183, Minority (label 0): 310
Number of 'good' native counterfactuals in data: 1125
Data distribution after iteration 1:
	Majority (1): 3183, Minority (label 0): 347
Number of 'good' native counterfactuals in data: 1136
Data distribution after iteration 2:
	Majority (1): 3183, Minority (label 0): 353
Number of 'good' native counterfactuals in data: 1141
Data distribution after iteration 3:
	Majority (1): 3183, Minority (label 0): 354
Number of 'good' native counterfactuals in data: 1141
Data distribution after iteration 4:
	Majority (1): 3183, Minority (label 0): 355
Number of 'good' native counterfactuals in data: 1141
Data distribution after iteration 5:
	Majority (1): 3183, Minority (label 0): 356
Number of 'good' native counterfactuals in data: 1141
Data distribution after iteration 6:
	Majority 

Data distribution after iteration 61:
	Majority (1): 3183, Minority (label 0): 412
Number of 'good' native counterfactuals in data: 1137
Data distribution after iteration 62:
	Majority (1): 3183, Minority (label 0): 413
Number of 'good' native counterfactuals in data: 1137
Data distribution after iteration 63:
	Majority (1): 3183, Minority (label 0): 414
Number of 'good' native counterfactuals in data: 1137
Data distribution after iteration 64:
	Majority (1): 3183, Minority (label 0): 415
Number of 'good' native counterfactuals in data: 1137
Data distribution after iteration 65:
	Majority (1): 3183, Minority (label 0): 416
Number of 'good' native counterfactuals in data: 1137
Data distribution after iteration 66:
	Majority (1): 3183, Minority (label 0): 417
Number of 'good' native counterfactuals in data: 1137
Data distribution after iteration 67:
	Majority (1): 3183, Minority (label 0): 418
Number of 'good' native counterfactuals in data: 1137
Data distribution after iteration 68:
	Ma

Number of 'good' native counterfactuals in data: 1133
Data distribution after iteration 122:
	Majority (1): 3183, Minority (label 0): 473
Number of 'good' native counterfactuals in data: 1133
Data distribution after iteration 123:
	Majority (1): 3183, Minority (label 0): 474
Number of 'good' native counterfactuals in data: 1133
Data distribution after iteration 124:
	Majority (1): 3183, Minority (label 0): 475
Number of 'good' native counterfactuals in data: 1133
Data distribution after iteration 125:
	Majority (1): 3183, Minority (label 0): 476
Number of 'good' native counterfactuals in data: 1133
Data distribution after iteration 126:
	Majority (1): 3183, Minority (label 0): 477
Number of 'good' native counterfactuals in data: 1133
Data distribution after iteration 127:
	Majority (1): 3183, Minority (label 0): 478
Number of 'good' native counterfactuals in data: 1133
Data distribution after iteration 128:
	Majority (1): 3183, Minority (label 0): 479
Number of 'good' native counterfac

Number of 'good' native counterfactuals in data: 1124
Data distribution after iteration 182:
	Majority (1): 3183, Minority (label 0): 533
Number of 'good' native counterfactuals in data: 1124
Data distribution after iteration 183:
	Majority (1): 3183, Minority (label 0): 534
Number of 'good' native counterfactuals in data: 1124
Data distribution after iteration 184:
	Majority (1): 3183, Minority (label 0): 535
Number of 'good' native counterfactuals in data: 1124
Data distribution after iteration 185:
	Majority (1): 3183, Minority (label 0): 536
Number of 'good' native counterfactuals in data: 1124
Data distribution after iteration 186:
	Majority (1): 3183, Minority (label 0): 537
Number of 'good' native counterfactuals in data: 1124
Data distribution after iteration 187:
	Majority (1): 3183, Minority (label 0): 538
Number of 'good' native counterfactuals in data: 1124
Data distribution after iteration 188:
	Majority (1): 3183, Minority (label 0): 539
Number of 'good' native counterfac

Number of 'good' native counterfactuals in data: 1093
Data distribution after iteration 242:
	Majority (1): 3183, Minority (label 0): 593
Number of 'good' native counterfactuals in data: 1093
Data distribution after iteration 243:
	Majority (1): 3183, Minority (label 0): 594
Number of 'good' native counterfactuals in data: 1093
Data distribution after iteration 244:
	Majority (1): 3183, Minority (label 0): 595
Number of 'good' native counterfactuals in data: 1093
Data distribution after iteration 245:
	Majority (1): 3183, Minority (label 0): 596
Number of 'good' native counterfactuals in data: 1093
Data distribution after iteration 246:
	Majority (1): 3183, Minority (label 0): 597
Number of 'good' native counterfactuals in data: 1092
Data distribution after iteration 247:
	Majority (1): 3183, Minority (label 0): 598
Number of 'good' native counterfactuals in data: 1092
Data distribution after iteration 248:
	Majority (1): 3183, Minority (label 0): 599
Number of 'good' native counterfac

...

Data distribution after iteration 697:
	Majority (1): 3173, Minority (label 0): 2556
Number of 'good' native counterfactuals in data: 716
Data distribution after iteration 698:
	Majority (1): 3173, Minority (label 0): 2559
Number of 'good' native counterfactuals in data: 716
Data distribution after iteration 699:
	Majority (1): 3173, Minority (label 0): 2562
Number of 'good' native counterfactuals in data: 716
Data distribution after iteration 700:
	Majority (1): 3173, Minority (label 0): 2565
Number of 'good' native counterfactuals in data: 716
Data distribution after iteration 701:
	Majority (1): 3173, Minority (label 0): 2568
Number of 'good' native counterfactuals in data: 716
Data distribution after iteration 702:
	Majority (1): 3173, Minority (label 0): 2571
Number of 'good' native counterfactuals in data: 716
Data distribution after iteration 703:
	Majority (1): 3173, Minority (label 0): 2574
Number of 'good' native counterfactuals in data: 716
Data distribution after iteration 

Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 760:
	Majority (1): 3173, Minority (label 0): 2745
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 761:
	Majority (1): 3173, Minority (label 0): 2748
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 762:
	Majority (1): 3173, Minority (label 0): 2751
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 763:
	Majority (1): 3173, Minority (label 0): 2754
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 764:
	Majority (1): 3173, Minority (label 0): 2757
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 765:
	Majority (1): 3173, Minority (label 0): 2760
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 766:
	Majority (1): 3173, Minority (label 0): 2763
Number of 'good' native counterfac

Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 820:
	Majority (1): 3173, Minority (label 0): 2925
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 821:
	Majority (1): 3173, Minority (label 0): 2928
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 822:
	Majority (1): 3173, Minority (label 0): 2931
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 823:
	Majority (1): 3173, Minority (label 0): 2934
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 824:
	Majority (1): 3173, Minority (label 0): 2937
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 825:
	Majority (1): 3173, Minority (label 0): 2940
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 826:
	Majority (1): 3173, Minority (label 0): 2943
Number of 'good' native counterfac

Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 880:
	Majority (1): 3173, Minority (label 0): 3105
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 881:
	Majority (1): 3173, Minority (label 0): 3108
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 882:
	Majority (1): 3173, Minority (label 0): 3111
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 883:
	Majority (1): 3173, Minority (label 0): 3114
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 884:
	Majority (1): 3173, Minority (label 0): 3117
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 885:
	Majority (1): 3173, Minority (label 0): 3120
Number of 'good' native counterfactuals in data: 747
Data distribution after iteration 886:
	Majority (1): 3173, Minority (label 0): 3123
Number of 'good' native counterfac

Data distribution after iteration 47:
	Majority (1): 3172, Minority (label 0): 387
Number of 'good' native counterfactuals in data: 995
Data distribution after iteration 48:
	Majority (1): 3172, Minority (label 0): 389
Number of 'good' native counterfactuals in data: 991
Data distribution after iteration 49:
	Majority (1): 3172, Minority (label 0): 391
Number of 'good' native counterfactuals in data: 991
Data distribution after iteration 50:
	Majority (1): 3172, Minority (label 0): 393
Number of 'good' native counterfactuals in data: 991
Data distribution after iteration 51:
	Majority (1): 3172, Minority (label 0): 395
Number of 'good' native counterfactuals in data: 990
Data distribution after iteration 52:
	Majority (1): 3172, Minority (label 0): 397
Number of 'good' native counterfactuals in data: 990
Data distribution after iteration 53:
	Majority (1): 3172, Minority (label 0): 399
Number of 'good' native counterfactuals in data: 990
Data distribution after iteration 54:
	Majority 

In [14]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'max_depth': 10, 'n_estimators': 600}
Best AUC: 0.7317163918516486


#### Approach 2: Verification of new synthetic counterfactuals with baseline model

In [11]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    print("Fitting baseline model needed for verification... ")
    baseline_search = GridSearchCV(RandomForestClassifier(random_state=19231823), rf_param_grid, scoring="roc_auc")
    baseline_search.fit(X_train, y_train)
    
    X_train_augmented, y_train_augmented = Iterative_CFA(X_train, 
                                                         y_train, 
                                                         stddev_percent=50, 
                                                         verify_with_baseline_model=True, 
                                                         baseline_model=baseline_search,
                                                         visualize_with_pca=False)
    
    for param_comb in combination_dicts:
        clf = RandomForestClassifier(random_state=19231823, **param_comb)
        clf.fit(X_train_augmented, y_train_augmented)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

Fold 0:
Fitting baseline model needed for verification... 
Data distribution before CFA:
	Majority (1): 3183, Minority (label 0): 158
Number of 'good' native counterfactuals in data: 1378
Classifier predicted all new synthetic counterfactuals to be in the majority class! => No new minority instances => Terminating...
Data distribution after CFA:
	Majority (1): 3183, Minority (label 0): 158
Fold 1:
Fitting baseline model needed for verification... 
Data distribution before CFA:
	Majority (1): 3185, Minority (label 0): 156
Number of 'good' native counterfactuals in data: 1056
Classifier predicted all new synthetic counterfactuals to be in the majority class! => No new minority instances => Terminating...
Data distribution after CFA:
	Majority (1): 3185, Minority (label 0): 156
Fold 2:
Fitting baseline model needed for verification... 
Data distribution before CFA:
	Majority (1): 3173, Minority (label 0): 169
Number of 'good' native counterfactuals in data: 1029
Classifier predicted all n

In [16]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'max_depth': 6, 'n_estimators': 600}
Best AUC: 0.7395650202089131


### ADASYN instead of CFA

In [17]:
from imblearn.over_sampling import ADASYN
ada = ADASYN(random_state=9317231)

In [ ]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    #print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        
    X_train_resampled, y_train_resampled = ada.fit_resample(X_train, y_train)
        
    for param_comb in combination_dicts:
        clf = RandomForestClassifier(random_state=19231823, **param_comb)
        clf.fit(X_train_resampled, y_train_resampled)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

In [19]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'max_depth': 10, 'n_estimators': 400}
Best AUC: 0.7172285092627817


### Random Oversampling

In [20]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=52012318)

In [ ]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    #print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
        
    for param_comb in combination_dicts:
        clf = RandomForestClassifier(random_state=19231823, **param_comb)
        clf.fit(X_train_resampled, y_train_resampled)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

In [22]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'max_depth': 6, 'n_estimators': 100}
Best AUC: 0.7175582926156095


## Logistic Regression 

In [12]:
from sklearn.linear_model import LogisticRegression

In [13]:
lr_param_grid = {"max_iter": [100, 200, 1000], 
                 "C": [0.001, 0.01, 0.1, 1, 10, 100, 1000],
                 "solver": ["newton-cg", "lbfgs", "liblinear", "sag"]}

# compute all combinations of parameters
combination_dicts = list(ParameterGrid(lr_param_grid))

### Baseline (no augmentation)

In [ ]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    #print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        
    for param_comb in combination_dicts:
        clf = LogisticRegression(random_state=19231823, **param_comb)
        clf.fit(X_train, y_train)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

In [26]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'C': 10, 'max_iter': 100, 'solver': 'liblinear'}
Best AUC: 0.7342425375116226


Again, we can compare to an out-of-the-box grid search to make sure:

In [27]:
baseline_search = GridSearchCV(LogisticRegression(random_state=19231823), lr_param_grid, scoring="roc_auc")
baseline_search.fit(X, y)
baseline_search.best_score_

C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linea

0.7366341749344969

### With CFA

#### Approach 1: No verification 

In [14]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    X_train_augmented, y_train_augmented = Iterative_CFA(X_train, 
                                                         y_train, 
                                                         stddev_percent=50, 
                                                         verify_with_baseline_model=False, 
                                                         visualize_with_pca=False)

    for param_comb in combination_dicts:
        clf = LogisticRegression(random_state=19231823, **param_comb)
        clf.fit(X_train_augmented, y_train_augmented)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

Fold 0:
Data distribution before CFA:
	Majority (1): 3183, Minority (label 0): 158
Number of 'good' native counterfactuals in data: 1378
Data distribution after iteration 0:
	Majority (1): 3183, Minority (label 0): 310
Number of 'good' native counterfactuals in data: 1125
Data distribution after iteration 1:
	Majority (1): 3183, Minority (label 0): 347
Number of 'good' native counterfactuals in data: 1136
Data distribution after iteration 2:
	Majority (1): 3183, Minority (label 0): 353
Number of 'good' native counterfactuals in data: 1141
Data distribution after iteration 3:
	Majority (1): 3183, Minority (label 0): 354
Number of 'good' native counterfactuals in data: 1141
Data distribution after iteration 4:
	Majority (1): 3183, Minority (label 0): 355
Number of 'good' native counterfactuals in data: 1141
Data distribution after iteration 5:
	Majority (1): 3183, Minority (label 0): 356
Number of 'good' native counterfactuals in data: 1141
Data distribution after iteration 6:
	Majority 

Number of 'good' native counterfactuals in data: 1137
Data distribution after iteration 65:
	Majority (1): 3183, Minority (label 0): 416
Number of 'good' native counterfactuals in data: 1137
Data distribution after iteration 66:
	Majority (1): 3183, Minority (label 0): 417
Number of 'good' native counterfactuals in data: 1137
Data distribution after iteration 67:
	Majority (1): 3183, Minority (label 0): 418
Number of 'good' native counterfactuals in data: 1137
Data distribution after iteration 68:
	Majority (1): 3183, Minority (label 0): 419
Number of 'good' native counterfactuals in data: 1137
Data distribution after iteration 69:
	Majority (1): 3183, Minority (label 0): 420
Number of 'good' native counterfactuals in data: 1137
Data distribution after iteration 70:
	Majority (1): 3183, Minority (label 0): 421
Number of 'good' native counterfactuals in data: 1137
Data distribution after iteration 71:
	Majority (1): 3183, Minority (label 0): 422
Number of 'good' native counterfactuals i

Number of 'good' native counterfactuals in data: 1133
Data distribution after iteration 131:
	Majority (1): 3183, Minority (label 0): 482
Number of 'good' native counterfactuals in data: 1133
Data distribution after iteration 132:
	Majority (1): 3183, Minority (label 0): 483
Number of 'good' native counterfactuals in data: 1133
Data distribution after iteration 133:
	Majority (1): 3183, Minority (label 0): 484
Number of 'good' native counterfactuals in data: 1133
Data distribution after iteration 134:
	Majority (1): 3183, Minority (label 0): 485
Number of 'good' native counterfactuals in data: 1133
Data distribution after iteration 135:
	Majority (1): 3183, Minority (label 0): 486
Number of 'good' native counterfactuals in data: 1133
Data distribution after iteration 136:
	Majority (1): 3183, Minority (label 0): 487
Number of 'good' native counterfactuals in data: 1133
Data distribution after iteration 137:
	Majority (1): 3183, Minority (label 0): 488
Number of 'good' native counterfac

Data distribution after iteration 190:
	Majority (1): 3183, Minority (label 0): 541
Number of 'good' native counterfactuals in data: 1124
Data distribution after iteration 191:
	Majority (1): 3183, Minority (label 0): 542
Number of 'good' native counterfactuals in data: 1124
Data distribution after iteration 192:
	Majority (1): 3183, Minority (label 0): 543
Number of 'good' native counterfactuals in data: 1124
Data distribution after iteration 193:
	Majority (1): 3183, Minority (label 0): 544
Number of 'good' native counterfactuals in data: 1124
Data distribution after iteration 194:
	Majority (1): 3183, Minority (label 0): 545
Number of 'good' native counterfactuals in data: 1124
Data distribution after iteration 195:
	Majority (1): 3183, Minority (label 0): 546
Number of 'good' native counterfactuals in data: 1124
Data distribution after iteration 196:
	Majority (1): 3183, Minority (label 0): 547
Number of 'good' native counterfactuals in data: 1099
Data distribution after iteration 

C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarnin

...

C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable

Fold 3:
Data distribution before CFA:
	Majority (1): 3172, Minority (label 0): 170
Number of 'good' native counterfactuals in data: 1031
Data distribution after iteration 0:
	Majority (1): 3172, Minority (label 0): 277
Number of 'good' native counterfactuals in data: 1027
Data distribution after iteration 1:
	Majority (1): 3172, Minority (label 0): 293
Number of 'good' native counterfactuals in data: 1032
Data distribution after iteration 2:
	Majority (1): 3172, Minority (label 0): 297
Number of 'good' native counterfactuals in data: 1033
Data distribution after iteration 3:
	Majority (1): 3172, Minority (label 0): 299
Number of 'good' native counterfactuals in data: 1031
Data distribution after iteration 4:
	Majority (1): 3172, Minority (label 0): 301
Number of 'good' native counterfactuals in data: 1031
Data distribution after iteration 5:
	Majority (1): 3172, Minority (label 0): 303
Number of 'good' native counterfactuals in data: 1030
Data distribution after iteration 6:
	Majority 

C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-l

Fold 4:
Data distribution before CFA:
	Majority (1): 3183, Minority (label 0): 159
Number of 'good' native counterfactuals in data: 1015
Data distribution after iteration 0:
	Majority (1): 3183, Minority (label 0): 279
Number of 'good' native counterfactuals in data: 1076
Data distribution after iteration 1:
	Majority (1): 3183, Minority (label 0): 292
Number of 'good' native counterfactuals in data: 1080
Data distribution after iteration 2:
	Majority (1): 3183, Minority (label 0): 293
Number of 'good' native counterfactuals in data: 1079
No unpaired instances found, no augmentation possible! Not applying CFA...
Data distribution after CFA:
	Majority (1): 3183, Minority (label 0): 293


C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-l

In [29]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'C': 1, 'max_iter': 100, 'solver': 'lbfgs'}
Best AUC: 0.626377917754169


#### Approach 2: Verification of new synthetic counterfactuals with baseline model

In [15]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
    
    print("Fitting baseline model needed for verification... ")
    baseline_search = GridSearchCV(LogisticRegression(random_state=19231823), lr_param_grid, scoring="roc_auc")
    baseline_search.fit(X_train, y_train)
    
    X_train_augmented, y_train_augmented = Iterative_CFA(X_train, 
                                                         y_train, 
                                                         stddev_percent=50, 
                                                         verify_with_baseline_model=True, 
                                                         baseline_model=baseline_search,
                                                         visualize_with_pca=False)

    for param_comb in combination_dicts:
        clf = LogisticRegression(random_state=19231823, **param_comb)
        clf.fit(X_train_augmented, y_train_augmented)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

Fold 0:
Fitting baseline model needed for verification... 


C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linea

Data distribution before CFA:
	Majority (1): 3183, Minority (label 0): 158
Number of 'good' native counterfactuals in data: 1378
Classifier predicted all new synthetic counterfactuals to be in the majority class! => No new minority instances => Terminating...
Data distribution after CFA:
	Majority (1): 3183, Minority (label 0): 158


C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable

Fold 1:
Fitting baseline model needed for verification... 


...

C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarnin

Fold 4:
Fitting baseline model needed for verification... 


C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linea

Data distribution before CFA:
	Majority (1): 3183, Minority (label 0): 159
Number of 'good' native counterfactuals in data: 1015
Classifier predicted all new synthetic counterfactuals to be in the majority class! => No new minority instances => Terminating...
Data distribution after CFA:
	Majority (1): 3183, Minority (label 0): 159


C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
C:\Users\fredd\Anaconda3\envs\cfa\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [31]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'C': 10, 'max_iter': 100, 'solver': 'liblinear'}
Best AUC: 0.7342425375116226


### ADASYN

In [ ]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    #print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        
    X_train_resampled, y_train_resampled = ada.fit_resample(X_train, y_train)
        
    for param_comb in combination_dicts:
        clf = LogisticRegression(random_state=19231823, **param_comb)
        clf.fit(X_train_resampled, y_train_resampled)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

In [33]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'C': 1, 'max_iter': 100, 'solver': 'liblinear'}
Best AUC: 0.7379872972484532


### Random Oversampling

In [ ]:
all_scores = []

for i, (train_index, test_index) in enumerate(kf.split(X)):
    #print(f"Fold {i}:")
    scores_for_fold = []
    X_train, X_test, y_train, y_test = X.iloc[train_index], X.iloc[test_index], y.iloc[train_index], y.iloc[test_index]
        
    X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)
        
    for param_comb in combination_dicts:
        clf = LogisticRegression(random_state=19231823, **param_comb)
        clf.fit(X_train_resampled, y_train_resampled)
        current_score = roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])
        #print(f"Score for combination {param_comb}: {current_score}")
        scores_for_fold.append(current_score)
        
    all_scores.append(scores_for_fold)

In [35]:
score_avg_over_folds = np.array(all_scores).mean(axis=0)
best_score_idx = score_avg_over_folds.argmax()
print(f"Best params: {combination_dicts[best_score_idx]}")
print(f"Best AUC: {score_avg_over_folds[best_score_idx]}")

Best params: {'C': 1, 'max_iter': 100, 'solver': 'liblinear'}
Best AUC: 0.7374792368165206
